In [13]:
import pandas as pd
import csv
from collections import OrderedDict

# input_file = csv.DictReader(open("datasets\data2.csv",encoding='cp850'))
input_file = csv.DictReader(open(r"C:\Users\Admin\Final_year_project\Final_year_ptoject\datasets\train2.csv",encoding = 'cp850'))
data = []
k = 0
# print(input_file)
for i in input_file:
#     print(dict(i))
#     print(k)
#     k = k + 1
    data.append(dict(i))

# data = pd.read_csv("datasets\data2.csv")
# print(type(d),d)

# data1 = pd.read_csv('datasets\data2.csv', header=None, index_col=0, squeeze=True).to_dict()
print(type(data),data,len(data))

<class 'list'> [{'text': 'Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all', 'target': '1'}, {'text': 'Forest fire near La Ronge Sask. Canada', 'target': '1'}, {'text': "All residents asked to 'shelter in place' are being notified by officers. No other evacuation or shelter in place orders are expected", 'target': '1'}, {'text': '13,000 people receive #wildfires evacuation orders in California ', 'target': '1'}, {'text': 'Just got sent this photo from Ruby #Alaska as smoke from #wildfires pours into a school ', 'target': '1'}, {'text': '#RockyFire Update => California Hwy. 20 closed in both directions due to Lake County fire - #CAfire #wildfires', 'target': '1'}, {'text': '#flood #disaster Heavy rain causes flash flooding of streets in Manitou, Colorado Springs areas', 'target': '1'}, {'text': "I'm on top of the hill and I can see a fire in the woods...", 'target': '1'}, {'text': "There's an emergency evacuation happening now in the building across the street", 'ta

In [16]:
import re
from nltk.tokenize import word_tokenize
from string import punctuation 
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords

class PreProcessTweets:
    def __init__(self):
        self._stopwords = set(stopwords.words('english') + list(punctuation) + ['AT_USER','URL'])
        
    def processTweets(self, list_of_tweets):
        processedTweets=[]
#         print("1")
#         print(list_of_tweets)
#         for i in list_of_tweets:
#             print(i["text"])
#             break
            
        for tweet in list_of_tweets:
#             print("22")
            processedTweets.append((self._processTweet(tweet["text"]),tweet["target"]))
#             print(processedTweets)
        return processedTweets
    
    def _processTweet(self, tweet):
        tweet = tweet.lower() # convert text to lower-case
        tweet = re.sub('((www\.[^\s]+)|(https?://[^\s]+))', 'URL', tweet) # remove URLs
        tweet = re.sub('@[^\s]+', 'AT_USER', tweet) # remove usernames
        tweet = re.sub(r'#([^\s]+)', r'\1', tweet) # remove the # in #hashtag
        tweet = word_tokenize(tweet) # remove repeated characters (helloooooooo into hello)
        return [word for word in tweet if word not in self._stopwords]

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [17]:
tweetProcessor = PreProcessTweets()
preprocessedTrainingSet = tweetProcessor.processTweets(data)

In [18]:
import nltk 

def buildVocabulary(preprocessedTrainingData):
    all_words = []
    
    for (words, sentiment) in preprocessedTrainingData:
        all_words.extend(words)

    wordlist = nltk.FreqDist(all_words)
    word_features = wordlist.keys()
    
    return word_features

In [19]:
def extract_features(tweet):
    tweet_words = set(tweet)
    features = {}
    for word in word_features:
        features['contains(%s)' % word] = (word in tweet_words)
    return features

In [20]:
word_features = buildVocabulary(preprocessedTrainingSet)
trainingFeatures = nltk.classify.apply_features(extract_features, preprocessedTrainingSet)

In [21]:
NBayesClassifier = nltk.NaiveBayesClassifier.train(trainingFeatures)

In [22]:
import pickle
f = open('nb1.pickle', 'wb')
pickle.dump(NBayesClassifier, f)
f.close()

In [23]:
import pickle
f = open('nb1.pickle', 'rb')
NBayesClassifier = pickle.load(f)
f.close()

In [27]:
import random
test = []
result = []
for i in range(100):
    t = random.choice(preprocessedTrainingSet)
#     print(t)
    test.append(t[0])
    result.append(t[1])
    

# print(test,result)

In [28]:
k = 0
p = 0
for i in test:

    NB = NBayesClassifier.classify(extract_features(i))
    if NB == result[k]:
        p = p + 1
    k = k + 1

print(p*100/k)


90.0


In [29]:
print(p*100/k)

90.0
